In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta

base_url = "https://index.golang.org/index?"

# Set the desired date for package retrieval and watch the magic happen
target_date = datetime(2023, 6, 11).date()

# Set the starting timestamp for the target date
start_time = datetime.combine(target_date, datetime.min.time())

# Initialize an empty DataFrame to store the data
all_data = pd.DataFrame()

while True:
    # Construct the API call URL for the current iteration
    current_url = base_url + "since=" + start_time.strftime("%Y-%m-%dT%H:%M:%SZ")
    
    # Make the API call
    response = requests.get(current_url)
    
    if response.status_code == 200:
        # Read the JSON response into a DataFrame
        data = pd.read_json(response.text, lines=True)
        
        # Filter out packages from subsequent days
        data = data[data["Timestamp"].dt.date == target_date]
        
        # Append the current data to the overall data
        all_data = all_data.append(data, ignore_index=True)
        
        # Check if there are more packages available
        if len(data) < 2000:
            break
        
        # Update the starting timestamp for the next iteration
        start_time = data["Timestamp"].iloc[-1] + timedelta(microseconds=1)
    else:
        print("Error occurred while making API call:", response.status_code)
        break

# Remove timezone from columns
all_data["Timestamp"] = all_data["Timestamp"].dt.tz_localize(None)

# Save the data as an Excel file
all_data.to_excel("output.xlsx", index=False)
